In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from pprint import pprint

sen_df: pd.DataFrame = pd.read_csv('/media/ponbac/BigHDD/ethereum/blacklist/seniority/seniority-tornado-result.csv', index_col=0)
poison_df: pd.DataFrame = pd.read_csv('/media/ponbac/BigHDD/ethereum/blacklist/poison/poison-tornado-result.csv', index_col=0)
haircut_df: pd.DataFrame = pd.read_csv('/media/ponbac/BigHDD/ethereum/blacklist/haircut/haircut-tornado-result.csv', index_col=0)
fifo_df: pd.DataFrame = pd.read_csv('/media/ponbac/BigHDD/ethereum/blacklist/fifo/fifo-tornado-result.csv', index_col=0)

# Fetch blacklisting results

In [ ]:
def get_seniority(address: str) -> float:
    '''
    Returns the seniority of an address.
    '''
    if address in seniority_df.index:
        return seniority_df.loc[address]['seniority']
    else:
        return 0

def get_poison(address: str) -> float:
    '''
    Returns the poison of an address.
    '''
    if address in poison_df.index:
        return poison_df.loc[address]['poison']
    else:
        return 0

def get_haircut(address: str) -> float:
    '''
    Returns the haircut of an address.
    '''
    if address in haircut_df.index:
        return haircut_df.loc[address]['haircut']
    else:
        return 0

def get_fifo(address: str) -> float:
    '''
    Returns the fifo of an address.
    '''
    if address in fifo_df.index:
        return fifo_df.loc[address]['fifo']
    else:
        return 0

# Fetch cluster results

In [ ]:
pprint("CLUSTERING DA STUFF")

# Combine the two into different factors

In [ ]:
pprint("Calcinack")

# Produce one final value!

In [ ]:
pprint("42!")